## Joins with `sqlalchemy`

To join to tables in `sqlalchemy`

1. Use `join` to create a `Join` object
2. Build a `select` statement from the `join` object

## Example - Reading in the Company `db`

In [1]:
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, func
from sqlalchemy.ext.automap import automap_base

engine = create_engine("sqlite:///databases/company_2_7_2.db")

Base = automap_base()
Base.prepare(engine, reflect=True)
Dept = Base.classes.department
Empl = Base.classes.employee

In [3]:
import pandas as pd
from sqlalchemy import select as selectq
d = selectq([Dept])
pd.read_sql_query(d, con=engine)

,DeptID,DeptName
0,31,Sales
1,33,Engineering
2,34,Clerical
3,35,Marketing


In [6]:
e = selectq([Empl])
pd.read_sql_query(e, con=engine)

,DeptID,LastName,EmpID
0,31.0,Rafferty,1
1,33.0,Jones,2
2,33.0,Heisenberg,3
3,34.0,Robinson,4
4,34.0,Smith,5
5,NaN,Williams,6


## Using `sqlalchemy.join` to create a `Join`

**Syntax:** `join(left_table, right_table, onclause=left_table.column == right_table.column)`

* Defaults to an inner join
* Set `isouter=True` to get a `LEFT OUTER JOIN`
* Set `full=True` to get a `FULL OUTER JOIN`

In [7]:
from sqlalchemy import join
j = join(Empl, Dept, onclause=Empl.DeptID == Dept.DeptID)
print(j)

employee JOIN department ON employee."DeptID" = department."DeptID"


## Inspecting the joined column names

Note that the columns are renamed `tableName_columnName`.

In [8]:
j.c.keys()

['employee_DeptID',
 'employee_LastName',
 'employee_EmpID',
 'department_DeptID',
 'department_DeptName']

## Creating a `Select` expression for a `Join`

In [9]:
from sqlalchemy import func, select as selectq

stmt = selectq([j])
print(stmt)

SELECT employee."DeptID", employee."LastName", employee."EmpID", department."DeptID", department."DeptName" 
FROM employee JOIN department ON employee."DeptID" = department."DeptID"


In [10]:
pd.read_sql_query(stmt, con=engine)

,DeptID,LastName,EmpID,DeptID,DeptName
0,31.0,Rafferty,1,31,Sales
1,33.0,Jones,2,33,Engineering
2,33.0,Heisenberg,3,33,Engineering
3,34.0,Robinson,4,34,Clerical
4,34.0,Smith,5,34,Clerical


## Left Join

In [11]:
left_join = join(Empl, Dept, onclause=Empl.DeptID==Dept.DeptID, isouter=True)
left_join_stmt = selectq([left_join])
pd.read_sql_query(left_join_stmt, con=engine)

,DeptID,LastName,EmpID,DeptID,DeptName
0,31.0,Rafferty,1,31.0,Sales
1,33.0,Jones,2,33.0,Engineering
2,33.0,Heisenberg,3,33.0,Engineering
3,34.0,Robinson,4,34.0,Clerical
4,34.0,Smith,5,34.0,Clerical
5,NaN,Williams,6,NaN,None


## Right Join

To get a `RIGHT OUTER JOIN`, just switch the order and use a `LEFT OUTER JOIN`

In [12]:
right_join = join(Dept, Empl, onclause=Empl.DeptID==Dept.DeptID, isouter=True)
right_join_stmt = selectq([right_join])
pd.read_sql_query(right_join_stmt, con=engine)

,DeptID,DeptName,DeptID,LastName,EmpID
0,31,Sales,31.0,Rafferty,1.0
1,33,Engineering,33.0,Heisenberg,3.0
2,33,Engineering,33.0,Jones,2.0
3,34,Clerical,34.0,Robinson,4.0
4,34,Clerical,34.0,Smith,5.0
5,35,Marketing,NaN,None,NaN


## Full Outer Join

**Note:** `sqllite` does not support this type of join `:/`

In [13]:
full_join = join(Empl, Dept, onclause=Empl.DeptID==Dept.DeptID, full=True)
full_join_stmt = selectq([full_join])
pd.read_sql_query(full_join_stmt, con=engine)

OperationalError: (sqlite3.OperationalError) RIGHT and FULL OUTER JOINs are not currently supported [SQL: 'SELECT employee."DeptID", employee."LastName", employee."EmpID", department."DeptID", department."DeptName" \nFROM employee FULL OUTER JOIN department ON employee."DeptID" = department."DeptID"'] (Background on this error at: http://sqlalche.me/e/e3q8)

## <font color="red"> Exercise 3 </font>

Determine all the players that have hit more than 100 home runs in a season.  The final table should include the players proper name, as well as the team name.  

**Hint:** You will need join the files listed below.  To get credit for this exercise, you will need to create a database containing these three tables and use the `sqlalchemy` join methods presented above.

In [102]:
f1, f2, f3 = ("./data/baseball/core/Batting.csv", 
              "./data/baseball/core/People.csv",
              "./data/baseball/core/Teams.csv")

# Your code here
import pandas as pd
batting = pd.read_csv(f1)
people = pd.read_csv(f2)
Teams = pd.read_csv(f3)

In [103]:
from dfply import *
df_with_index_as_key = lambda df_no_key :(df_no_key>>
                        mutate(id = df_no_key.index))
batting = df_with_index_as_key(batting)
people = df_with_index_as_key(people)
teams = df_with_index_as_key(Teams)

In [104]:
!rm ./databases/baseball.db
from sqlalchemy import create_engine
engine = create_engine('sqlite:///databases/baseball.db', echo=False)

from sqlalchemy import Integer, Float, String
import numpy as np

def replace_V(v):
    if str(v) == 'object': 
        v = String
    if str(v) == 'int64': 
        v = Integer
    if str(v) == 'float64': 
        v = Float
    return v
    
        

SQL_batting_dtypes = {k:replace_V(v) for k,v in zip(batting.columns,batting.dtypes)}


schema = pd.io.sql.get_schema(batting, # dataframe
                              'batting', # name in SQL db
                              keys='id', # primary key
                              con=engine, # connection
                              dtype=SQL_batting_dtypes # SQL types
                             )


engine.execute(schema)

batting.to_sql('batting', 
                  con=engine, 
                  dtype=SQL_batting_dtypes, 
                  index=False,
                  if_exists='append')

rm: ./databases/baseball.db: No such file or directory


In [105]:

SQL_people_dtypes = {k:replace_V(v) for k,v in zip(people.columns,people.dtypes)}


schema = pd.io.sql.get_schema(people, # dataframe
                              'people', # name in SQL db
                              keys='id', # primary key
                              con=engine, # connection
                              dtype=SQL_people_dtypes # SQL types
                             )
engine.execute(schema)

people.to_sql('people', 
                  con=engine, 
                  dtype=SQL_people_dtypes, 
                  index=False,
                  if_exists='append')

In [106]:

SQL_teams_dtypes = {k:replace_V(v) for k,v in zip(teams.columns,teams.dtypes)}


schema = pd.io.sql.get_schema(teams, # dataframe
                              'teams', # name in SQL db
                              keys='id', # primary key
                              con=engine, # connection
                              dtype=SQL_teams_dtypes # SQL types
                             )
engine.execute(schema)

teams.to_sql('teams', 
                  con=engine, 
                  dtype=SQL_teams_dtypes, 
                  index=False,
                  if_exists='append')

In [107]:
from sqlalchemy import create_engine
from sqlalchemy.ext.automap import automap_base
Base = automap_base()
engine2 = create_engine('sqlite:///databases/baseball.db')
engine2.echo = False
Base.prepare(engine2, reflect=True)
Batting = Base.classes.batting
People = Base.classes.people
Teams = Base.classes.teams

In [109]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine2)
session = Session()
from sqlalchemy import select as selectq
stmt = selectq('*').select_from(Batting)
from more_sqlalchemy import result_dicts

In [110]:
batting = session.execute(stmt).fetchmany(1)>>result_dicts
batting

[{'playerID': 'abercda01',
  'yearID': 1871,
  'stint': 1,
  'teamID': 'TRO',
  'lgID': None,
  'G': 1,
  'AB': 4,
  'R': 0,
  'H': 0,
  '2B': 0,
  '3B': 0,
  'HR': 0,
  'RBI': 0.0,
  'SB': 0.0,
  'CS': 0.0,
  'BB': 0,
  'SO': 0.0,
  'IBB': None,
  'HBP': None,
  'SH': None,
  'SF': None,
  'GIDP': 0.0,
  'id': 0}]

In [112]:
from sqlalchemy import join
left_join = join(People, Batting, onclause=People.playerID==Batting.playerID, isouter=True)
left_join_stmt = selectq([left_join])
batting_people = pd.read_sql_query(left_join_stmt, con=engine)

In [117]:
SQL_BP_dtypes = {k:replace_V(v) for k,v in zip(batting_people.columns,batting_people.dtypes)}


schema = pd.io.sql.get_schema(batting_people, # dataframe
                              'batting_people', # name in SQL db
                              keys='id', # primary key
                              con=engine, # connection
                              dtype=SQL_BP_dtypes # SQL types
                             )
engine.execute(schema)

batting_people.to_sql('batting_people', 
                  con=engine, 
                  dtype=SQL_BP_dtypes, 
                  index=False,
                  if_exists='append')

IntegrityError: (sqlite3.IntegrityError) NOT NULL constraint failed: batting_people.id [SQL: 'INSERT INTO batting_people ("playerID", "birthYear", "birthMonth", "birthDay", "birthCountry", "birthState", "birthCity", "deathYear", "deathMonth", "deathDay", "deathCountry", "deathState", "deathCity", "nameFirst", "nameLast", "nameGiven", weight, height, bats, throws, debut, "finalGame", "retroID", "bbrefID", id, "yearID", stint, "teamID", "lgID", "G", "AB", "R", "H", "2B", "3B", "HR", "RBI", "SB", "CS", "BB", "SO", "IBB", "HBP", "SH", "SF", "GIDP") VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)'] [parameters: (('aardsda01', 1981.0, 12.0, 27.0, 'USA', 'CO', 'Denver', None, None, None, None, None, None, 'David', 'Aardsma', 'David Allan', 215.0, 75.0, 'R', 'R', '2004-04-06', '2015-08-23', 'aardd001', 'aardsda01', 84655.0, 2004.0, 1.0, 'SFN', 'NL', 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

In [115]:
right_join = join(Teams,batting_people, onclause=Teams.yearID==batting_people.playerID, isouter=True)
right_join_stmt = selectq([right_join]).select_from()
batting_people_teams = pd.read_sql_query(right_join_stmt, con=engine)

NameError: name 'Batting_people' is not defined

## Up Next

Stuff